#Exercice1 : Création d'un Modèle de Langage avec un Simple RNN et Keras
## Objectifs :
1. Télécharger un corpus textuel et le préparer pour la modélisation.
2. Représenter chaque mot avec un vecteur one-hot.
3. Construire un modèle RNN simple pour entraîner un modèle de langage.
4. Tester le modèle pour prédire le mot suivant et estimer la probabilité d’une phrase.

## Étape 1 : Téléchargement et Prétraitement des Données

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [49]:
# Étape 1 : Chargement et prétraitement des données
# Téléchargement d'un corpus texte
corpus = """Nous créons un modèle de langage basé sur un réseau de neurones récurrents.
            Ce modèle peut prédire le mot suivant dans une phrase et donner des probabilités
            pour des phrases entières."""

# Tokenization : transformer le texte en une liste de mots
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}  # Dictionnaire inverse

# Convertir le texte en séquences d'index
sequences = tokenizer.texts_to_sequences([corpus])[0]
vocab_size = len(word_index) + 1  # Ajouter 1 pour le padding éventuel

# Création de données d'entraînement (X, y)
X = []
y = []

for i in range(1, len(sequences)):
    X.append(sequences[:i])  # Historique des mots
    y.append(sequences[i])   # Mot suivant

# Padding des séquences pour avoir des longueurs fixes
max_sequence_len = max([len(x) for x in X])
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_sequence_len, padding='pre')

# Convertir les labels (y) en one-hot
y = to_categorical(y, num_classes=vocab_size)

## Étape 2 : Création du Modèle RNN

In [50]:
# Étape 2 : Création du modèle
model = Sequential([
    Embedding(vocab_size, 10, input_length=max_sequence_len),  # Embedding pour mapper les mots
    SimpleRNN(64, activation='tanh'),  # Couche RNN
    Dense(vocab_size, activation='softmax')  # Prédiction du prochain mot
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#  Entraînement du modèle
model.fit(X, y, epochs=200, verbose=2)


Epoch 1/200
1/1 - 2s - 2s/step - accuracy: 0.0000e+00 - loss: 3.3261
Epoch 2/200
1/1 - 0s - 25ms/step - accuracy: 0.0667 - loss: 3.2870
Epoch 3/200
1/1 - 0s - 57ms/step - accuracy: 0.1333 - loss: 3.2495
Epoch 4/200
1/1 - 0s - 23ms/step - accuracy: 0.2000 - loss: 3.2096
Epoch 5/200
1/1 - 0s - 57ms/step - accuracy: 0.2333 - loss: 3.1655
Epoch 6/200
1/1 - 0s - 23ms/step - accuracy: 0.2333 - loss: 3.1172
Epoch 7/200
1/1 - 0s - 56ms/step - accuracy: 0.2000 - loss: 3.0668
Epoch 8/200
1/1 - 0s - 56ms/step - accuracy: 0.2333 - loss: 3.0172
Epoch 9/200
1/1 - 0s - 23ms/step - accuracy: 0.2333 - loss: 2.9626
Epoch 10/200
1/1 - 0s - 23ms/step - accuracy: 0.2667 - loss: 2.9075
Epoch 11/200
1/1 - 0s - 23ms/step - accuracy: 0.2667 - loss: 2.8628
Epoch 12/200
1/1 - 0s - 23ms/step - accuracy: 0.3000 - loss: 2.8154
Epoch 13/200
1/1 - 0s - 26ms/step - accuracy: 0.2667 - loss: 2.7594
Epoch 14/200
1/1 - 0s - 23ms/step - accuracy: 0.3333 - loss: 2.7160
Epoch 15/200
1/1 - 0s - 22ms/step - accuracy: 0.3333 - 

## Étape 3 : Tester le Modèle
### Code : Prédire le Mot Suivant

In [10]:
def predict_next_word(model, tokenizer, input_text, max_sequence_len):
    """
    Prédit le mot suivant pour une séquence donnée.
    """
    input_seq = tokenizer.texts_to_sequences([input_text])[0]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=max_sequence_len, padding='pre')
    predicted = np.argmax(model.predict(input_seq), axis=-1)
    return tokenizer.index_word[predicted[0]]

# Exemple 1 de prédiction
input_text = "Nous créons un modèle de"
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Phrase initiale : '{input_text}'")
print(f"Mot prédit : '{next_word}'")
# Exemple 2 de prédiction
input_text = "Nous construisons un modèle de"
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Phrase initiale : '{input_text}'")
print(f"Mot prédit : '{next_word}'")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Phrase initiale : 'Nous créons un modèle de'
Mot prédit : 'langage'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Phrase initiale : 'Nous construisons un modèle de'
Mot prédit : 'de'


### Code : Calculer la Probabilité d’une Phrase

In [ ]:
def compute_sentence_probability(model, tokenizer, sentence, max_sequence_len):
    """
    Calcule la probabilité totale d'une phrase.
    """
    sentence_seq = tokenizer.texts_to_sequences([sentence])[0]
    prob = 1.0
    for i in range(1, len(sentence_seq)):
        input_seq = tf.keras.preprocessing.sequence.pad_sequences([sentence_seq[:i]], maxlen=max_sequence_len, padding='pre')
        predicted_probs = model.predict(input_seq)
        word_prob = predicted_probs[0][sentence_seq[i]]
        prob *= word_prob
    return prob

# Exemple : Probabilité d'une phrase
sentence = "Nous créons un modèle"
prob = compute_sentence_probability(model, tokenizer, sentence, max_sequence_len)
print(f"Phrase : '{sentence}'")
print(f"Probabilité : {prob}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Phrase : 'Nous créons un modèle'
Probabilité : 0.11122305726755227


# EXercice 2: Entraînement et Test d'un Modèle Word2Vec
L'objectif de ce TP est de comprendre et d'implémenter un modèle Word2Vec pour apprendre les représentations vectorielles de mots (word embeddings) à partir d'un corpus textuel. Vous utiliserez la bibliothèque gensim pour entraîner un modèle Word2Vec, puis testerez les embeddings générés à travers diverses analyses telles que la recherche des mots similaires et les analogies.

In [12]:
import nltk
import gensim
from nltk.corpus import abc
nltk.download('punkt')
import gensim.downloader as api
corpus = api.load('text8')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
model= gensim.models.Word2Vec(corpus, workers=4)

In [6]:
X = list(model.wv.key_to_index.keys())
data=model.wv['science']
print(data)

[ 0.23585504 -0.4868843   0.73778105  0.27229264  0.35432124  0.9800996
 -1.7709215   0.13693951  2.7570286  -0.9942347   0.5860272  -2.8883467
 -0.7176219  -1.3383068  -2.660156    2.4224882  -2.4666743  -0.8131905
 -1.750751   -0.39751908 -1.1187401   1.272431   -0.2520156  -0.79906744
 -0.5683308   0.5270443   2.2557502  -1.9209225  -0.78695375  0.6719341
  0.29028213 -1.9436154   0.8765284   1.2289647   0.5062359  -2.081198
  0.43020558  2.220376    0.5565584  -2.7431839  -0.16032003 -3.6848242
  1.3996514   0.9480189   0.06678706 -1.1434996   0.6670449   0.25639382
 -0.37875807 -2.0502992   3.4607916   0.7636802  -2.3933957  -2.0550563
 -0.42045432  0.520604    1.9334751  -0.876951    1.2579758  -0.47389975
 -0.39349097 -0.994491    0.5972023  -1.142644    1.3027773   1.9590539
  3.6296976  -1.5109603   1.4837697   2.4429612  -3.4985023   1.6454178
 -0.46803385  0.03718194 -1.7377222  -0.19856717  1.6633523   0.23352091
  1.1030923  -0.28877056 -0.604741   -0.51883554 -0.82935673 

In [7]:
len(X)

71290

In [8]:
model.wv.most_similar('science')

[('psychology', 0.6578212976455688),
 ('anthropology', 0.6454300284385681),
 ('sociology', 0.6031381487846375),
 ('sciences', 0.6028887033462524),
 ('humanities', 0.5975008606910706),
 ('philosophy', 0.5932278037071228),
 ('memetics', 0.5889267325401306),
 ('astronomy', 0.5817707180976868),
 ('zoology', 0.5812671184539795),
 ('metaphysics', 0.5717838406562805)]

In [9]:
model.wv.most_similar('man')

[('woman', 0.7208858132362366),
 ('girl', 0.6718136072158813),
 ('creature', 0.5929964780807495),
 ('stranger', 0.5894483327865601),
 ('boy', 0.5893861651420593),
 ('god', 0.5739547610282898),
 ('lover', 0.5588944554328918),
 ('gentleman', 0.556574285030365),
 ('evil', 0.5535324215888977),
 ('person', 0.5527227520942688)]

In [10]:
model.wv.most_similar('english')

[('welsh', 0.6402224898338318),
 ('irish', 0.6375089883804321),
 ('afrikaans', 0.5895422101020813),
 ('gaelic', 0.5837380886077881),
 ('scottish', 0.5816424489021301),
 ('hindi', 0.5793278217315674),
 ('icelandic', 0.5655657052993774),
 ('esperanto', 0.5647801756858826),
 ('dutch', 0.5641345381736755),
 ('french', 0.5639718770980835)]

In [11]:
model.wv.most_similar('paris')

[('venice', 0.7488073110580444),
 ('vienna', 0.7186405062675476),
 ('munich', 0.7050175070762634),
 ('milan', 0.7024926543235779),
 ('bologna', 0.6906524300575256),
 ('florence', 0.6811869740486145),
 ('louvre', 0.6811328530311584),
 ('leipzig', 0.6747598052024841),
 ('naples', 0.664282500743866),
 ('france', 0.6623011827468872)]

In [12]:
model.wv.most_similar(positive=['king', 'woman'], negative=['man'],topn=1)

[('queen', 0.6441126465797424)]

In [13]:
model.wv.doesnt_match(['psychology','mathematics','medicine','orange'])

'orange'

#Exercice 3: Création d'un Modèle de Langage avec Intégration de Word2Vec

In [47]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical
import gensim.downloader as api

# Étape 1 : Charger un corpus pré-entraîné pour Word2Vec
#corpus = api.load('text8')  # Chargement d'un corpus Word2Vec
#model = gensim.models.Word2Vec(corpus, vector_size=100, workers=4)  # Word2Vec

# Extraire le vocabulaire et les embeddings de Word2Vec
word_vectors = model.wv
embedding_dim = word_vectors.vector_size
vocab_w2v = word_vectors.key_to_index  # Vocabulaire Word2Vec

# Étape 2 : Préparer un corpus texte pour le RNN
text_corpus = """We create a language model based on a recurrent neural network.
                 This model can predict the next word in a sentence
                 and give probabilities for entire sentences."""
sentences = [sentence.split() for sentence in text_corpus.split('\n')]

# Tokenization du texte
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(sentence) for sentence in sentences])
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}  # Dictionnaire inverse
vocab_size = len(word_index) + 1  # Taille du vocabulaire

# Étape 3 : Créer une matrice d'embedding basée sur Word2Vec
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in vocab_w2v:  # Vérifier si le mot existe dans Word2Vec
        embedding_matrix[i] = word_vectors[word]

# Étape 4 : Convertir le corpus en données d'entraînement
sequences = tokenizer.texts_to_sequences([' '.join(sentence) for sentence in sentences])

# Construire X et y (historique des mots et mot suivant)
X, y = [], []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

# Padding des séquences
max_sequence_len = max([len(x) for x in X])
X = pad_sequences(X, maxlen=max_sequence_len, padding='pre')
y = to_categorical(y, num_classes=vocab_size)

# Étape 5 : Créer le modèle RNN
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_sequence_len,
              trainable=False),  # Couche embedding initialisée avec Word2Vec
    SimpleRNN(128, activation='tanh'),
    Dense(vocab_size, activation='softmax')  # Prédiction du mot suivant
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Étape 6 : Entraîner le modèle
model.fit(X, y, epochs=10, batch_size=64, verbose=2)



AttributeError: 'Sequential' object has no attribute 'wv'

In [17]:
# Étape 7 : Prédire le mot suivant
def predict_next_word(model, tokenizer, input_text, max_sequence_len):
    """
    Prédit le mot suivant pour une séquence donnée.
    """
    input_seq = tokenizer.texts_to_sequences([input_text])[0]
    input_seq = pad_sequences([input_seq], maxlen=max_sequence_len, padding='pre')
    predicted_index = np.argmax(model.predict(input_seq), axis=-1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word
    return None


# Exemple 1 : Prédiction du mot suivant
input_text = "We create a language"
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Phrase initiale : '{input_text}'")
print(f"Mot prédit : '{next_word}'")

# Exemple 2 : Prédiction du mot suivant
input_text = "We construct a language"
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Phrase initiale : '{input_text}'")
print(f"Mot prédit : '{next_word}'")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
Phrase initiale : 'We create a language'
Mot prédit : 'model'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Phrase initiale : 'We construct a language'
Mot prédit : 'model'


In [16]:
word_vectors.most_similar('create')

[('introduce', 0.7879522442817688),
 ('adapt', 0.7436988949775696),
 ('develop', 0.7314819693565369),
 ('produce', 0.7211201190948486),
 ('construct', 0.7186512351036072),
 ('build', 0.7149459719657898),
 ('generate', 0.7092704176902771),
 ('enhance', 0.7090157270431519),
 ('establish', 0.7062076926231384),
 ('incorporate', 0.6981716156005859)]

In [18]:
# Étape 8 : Calculer la probabilité d'une phrase
def compute_sentence_probability(model, tokenizer, sentence, max_sequence_len):
    """
    Calcule la probabilité totale d'une phrase.
    """
    sentence_seq = tokenizer.texts_to_sequences([sentence])[0]
    prob = 1.0
    for i in range(1, len(sentence_seq)):
        input_seq = pad_sequences([sentence_seq[:i]], maxlen=max_sequence_len, padding='pre')
        predicted_probs = model.predict(input_seq)
        word_prob = predicted_probs[0][sentence_seq[i]]
        prob *= word_prob
    return prob


# Exemple : Probabilité d'une phrase
sentence = "We create a language model"
prob = compute_sentence_probability(model, tokenizer, sentence, max_sequence_len)
print(f"Phrase : '{sentence}'")
print(f"Probabilité : {prob}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Phrase : 'We create a language model'
Probabilité : 0.023096473106229542


# Exercice 4 :Création d'un Modèle d'Analyse de Sentiment avec LSTM et Word2Vec
L'analyse de sentiment est un problème classique en traitement automatique du langage naturel (TALN) où le but est de déterminer si un texte exprime une opinion positive ou négative. Dans ce TP, vous utiliserez un corpus annoté contenant des avis (par exemple, des avis de films) pour entraîner un modèle basé sur une architecture LSTM.

Les étapes principales incluent :

1. Prétraitement des données textuelles.
2. Utilisation de Word2Vec pour générer des vecteurs d'embedding pour les mots du corpus.
3. Construction d'une architecture LSTM pour capturer les dépendances séquentielles dans les textes.
4. Entraînement et évaluation du modèle sur des données annotées.

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import gensim.downloader as api

In [29]:
# Étape 1 : Charger et entraîner le modèle Word2Vec
print("Chargement et entraînement de Word2Vec...")
corpus = api.load('text8')  # Charger un corpus de texte pré-entraîné
word2vec = gensim.models.Word2Vec(corpus, vector_size=100, workers=4)  # Taille de l'embedding = 100
word_vectors = word2vec.wv  # Récupérer les vecteurs de mots
vocab_size = len(word_vectors)  # Taille du vocabulaire
print(f"Word2Vec entraîné avec un vocabulaire de {vocab_size} mots.")


Chargement et entraînement de Word2Vec...
Word2Vec entraîné avec un vocabulaire de 71290 mots.


In [30]:
# Étape 2 : Prétraitement des données IMDB
print("Prétraitement des données IMDB...")
max_features = 10000  # Limite des mots les plus fréquents
maxlen = 500  # Longueur des séquences

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Ajouter des zéros pour aligner toutes les séquences à maxlen
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


Prétraitement des données IMDB...


In [31]:
# Étape 3 : Créer une matrice d'embedding basée sur Word2Vec
print("Création de la matrice d'embedding Word2Vec...")
embedding_dim = 100
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, index in imdb.get_word_index().items():
    if index < max_features:
        try:
            embedding_vector = word_vectors[word]
            embedding_matrix[index] = embedding_vector
        except KeyError:
            continue  # Si le mot n'existe pas dans Word2Vec, on le laisse à zéro

Création de la matrice d'embedding Word2Vec...
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [46]:
# Étape 4 : Construire le modèle LSTM
print("Construction du modèle LSTM...")
model = Sequential([
    Embedding(input_dim=max_features,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=maxlen,
              trainable=False),  # Embedding fixe
    LSTM(128, return_sequences=True),  # Couche LSTM avec 128 unités
    Dropout(0.3),  # Régularisation
    LSTM(64, return_sequences=False),  # Couche LSTM avec 128 unités
    Dropout(0.3),  # Régularisation
    Dense(1, activation='sigmoid')  # Prédiction binaire (positif/négatif)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Construction du modèle LSTM...


In [43]:
# Étape 5 : Entraîner le modèle
print("Entraînement du modèle...")
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.2, verbose=2)

Entraînement du modèle...
Epoch 1/5
157/157 - 11s - 67ms/step - accuracy: 0.6250 - loss: 0.6392 - val_accuracy: 0.7104 - val_loss: 0.5629
Epoch 2/5
157/157 - 10s - 64ms/step - accuracy: 0.7271 - loss: 0.5443 - val_accuracy: 0.7390 - val_loss: 0.5274
Epoch 3/5
157/157 - 8s - 52ms/step - accuracy: 0.7767 - loss: 0.4763 - val_accuracy: 0.6502 - val_loss: 0.6443
Epoch 4/5
157/157 - 11s - 72ms/step - accuracy: 0.7492 - loss: 0.5167 - val_accuracy: 0.7780 - val_loss: 0.4724
Epoch 5/5
157/157 - 10s - 62ms/step - accuracy: 0.7953 - loss: 0.4467 - val_accuracy: 0.8036 - val_loss: 0.4361


In [44]:
# Étape 6 : Évaluer le modèle
print("Évaluation sur les données de test...")
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Précision sur les données de test : {accuracy:.2f}")

Évaluation sur les données de test...
Précision sur les données de test : 0.81


In [39]:
# Étape 7 : Tester le modèle avec un exemple
print("Prédiction sur un exemple...")
word_index = imdb.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}

def decode_review(sequence):
    return " ".join([reverse_word_index.get(i - 3, "?") for i in sequence])

sample_index = 1
sample_review = x_test[sample_index]
predicted_sentiment = model.predict(sample_review.reshape(1, -1))[0][0]
decoded_review = ' '.join(decoded_review.replace('?', '').split())
print(f"Avis : {decoded_review.replace('?', '')}")
print(f"Sentiment prédit : {'Positif' if predicted_sentiment > 0.5 else 'Négatif'} (score : {predicted_sentiment:.2f})")


Prédiction sur un exemple...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Avis : this film requires a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere with sexual tension and psychological it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy has a small part the moody set fits the content of the story very well in short this movie is a powerful study of loneliness sexual a